In [1]:
import numpy as np
import pandas as pd
import os
import shutil
from PIL import Image
import torch
from tqdm import tqdm
from distutils.dir_util import copy_tree

In [2]:
WORKING_DIR = ''

# Predict

In [6]:
model = torch.hub.load('ultralytics/yolov5', 'custom', path=os.path.join(WORKING_DIR, 'best.pt'))

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2022-10-22 Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
YOLOv5l summary: 267 layers, 46129818 parameters, 0 gradients
Adding AutoShape... 


In [7]:
model.conf = 0.7

In [8]:
classes = ['3.20', '2.1', '2.4', '3.1', '1.25']

In [9]:
sings = ['3.24', '1.16', '5.15.5', '5.19.1', '5.19.2', '1.20.1', '8.23',
'2.1', '4.2.1', '8.22.1', '6.16', '1.22', '1.2', '5.16', '3.27',
'6.10.1', '8.2.4', '6.12', '5.15.2', '3.13', '3.1', '3.20', '3.12',
'7.14.2', '5.23.1', '2.4', '5.6', '4.2.3', '8.22.3', '5.15.1',
'7.3', '3', '2.3.1', '3.11', '6.13', '5.15.4', '8.2.1', '1.34.3',
'8.2.2', '5.15.3', '1.17', '4.1.1', '4.1.4', '3.25', '1.20.2',
'8.22.2', '6.9.2', '3.2', '5.5', '5.15.7', '7.12', '8.2.3',
'5.24.1', '1.25', '3.28', '5.9.1', '5.15.6', '8.1.1', '1.10',
'6.11', '3.4', '6.10', '6.9.1', '8.2.5', '5.15', '4.8.2', '8.22',
'5.21', '5.18']

In [15]:
test = pd.read_csv(os.path.join(WORKING_DIR, 'test.csv'))

In [16]:
df = pd.DataFrame({}, columns=['id', 'sing1', 'sing2', 'sing3', 'sing4', 'sing5', 'sing6', 'sing7', 'sing8'])

In [20]:
for i in tqdm(range(len(test))):
    img_id, img_name = test.iloc[i]
    img = Image.open(os.path.join(WORKING_DIR, 'test/', img_name))
    preds = model(img).xyxy[0].tolist()
    if len(preds) == 0:
        df.loc[len(df)] = [img_id] + [0] * 8
        continue
    class_ids = []
    for _, _, _, _, conf, class_id in preds:
        class_id = int(class_id)
        class_name = classes[class_id]
        if class_name not in sings:
            continue
        class_id = sings.index(class_name) + 1
        class_ids.append(class_id)
    class_ids = class_ids[:8]
    class_ids = class_ids + [0] * (8 - len(class_ids))
    df.loc[len(df)] = [img_id] + class_ids

100%|██████████| 388/388 [02:31<00:00,  2.57it/s]


In [21]:
df.to_csv('out.csv', index=False)

In [22]:
df

,id,sing1,sing2,sing3,sing4,sing5,sing6,sing7,sing8
0,754,22,0,0,0,0,0,0,0
1,29,0,0,0,0,0,0,0,0
2,1157,8,0,0,0,0,0,0,0
3,1049,0,0,0,0,0,0,0,0
4,715,8,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
383,243,0,0,0,0,0,0,0,0
384,921,8,0,0,0,0,0,0,0
385,881,8,0,0,0,0,0,0,0
386,503,8,0,0,0,0,0,0,0
